# [Graphs and Sessions](https://www.tensorflow.org/programmers_guide/graphs)

TensorFlow uses a **dataflow graph** to represent your computation in terms of the dependencies between individual operations.

Dataflow has several advantages that TensorFlow leverages when executing your programs:
- **Parallelism.** easy for the system to identify operations that can execute in parallel.
- **Distributed execution.** possible for TensorFlow to partition your program across multiple devices (CPUs, GPUs and TPUs) attached to different machines.
- **Compilation.** TensorFlow's XLA compiler can use the information in your dataflow graph to generate faster code.
- **Portability.** The dataflow graph is a language-independent representation of the code in your model.

## tf.Graph
A ```tf.Graph``` contains two relevent kinds of information:
- Graph structure. Nodes and edges indicate how individual operations are composed together.
- Graph collections. A general mechanism for storing collections of metadata in a ```tf.Graph```.
  - ```tf.add_to_collection``` enables you to associate a list of objects with a key.
  - ```tf.GraphKeys``` defines some of the standard keys.
  - ```tf.get_collection``` enables you to look up all objects associated with a key.

### Building a ```tf.Graph```
Most TensorFlow programs start with a dataflow graph construction phase constructing new ```tf.Operation``` (node) and ```tf.Tensor``` (edge) objects and adding them to a ```tf.Graph```. For example:
- Calling ```tf.train.Optimizer.minimize``` will add operations and tensors to the default graph that calculate gradients, and return a ```tf.Operation``` that, when run, will apply those gradients to a set of variables.

TensorFlow provides a default graph that is an implicit argument to all API functions in the same context. Tensorflow also allows multiple graphs for more advanced use cases.

### Naming operations
A ```tf.Graph``` object defines a namespace for the ```tf.Operation``` objects it contains. TensorFlow automatically chooses a unique name for each operation in graph if name is not given explicitly. The TensorFlow API provides two ways to override the name of an operation:
- Each API function that creates a new ```tf.Operation``` or returns a new ```tf.Tensor``` accepts an optional name argument. 
  - For example, ```tf.constant(42.0, name="answer")``` creates a new ```tf.Operation``` named "answer" and returns a ```tf.Tensor``` named "answer:0". If the graph already uses "answer:0", 
- The ```tf.name_scope``` function makes it possible to add a name scope prefix to all operations created in a particular context. For example:
  ```python
  c_0 = tf.constant(0, name="c")  # => operation named "c"
  # Already-used names will be "uniquified".
  c_1 = tf.constant(2, name="c")  # => operation named "c_1"
  # Name scopes add a prefix to all operations created in the same context.
  with tf.name_scope("outer"):
    c_2 = tf.constant(2, name="c")  # => operation named "outer/c"
    # Name scopes nest like paths in a hierarchical file system.
    with tf.name_scope("inner"):
      c_3 = tf.constant(3, name="c")  # => operation named "outer/inner/c"
    # Exiting a name scope context will return to the previous prefix.
    c_4 = tf.constant(4, name="c")  # => operation named "outer/c_1"
    # Already-used name scopes will be "uniquified".
    with tf.name_scope("inner"):
      c_5 = tf.constant(5, name="c")  # => operation named "outer/inner_1/c"
  ```
Note that ```tf.Tensor``` objects are implicitly named after the ```tf.Operation``` that produces the tensor as output. A tensor name has the form ```"<OP_NAME>:<i>"``` where:
- ```"<OP_NAME>"``` is the name of the operation that produces it.
- ```"<i>"``` is an integer representing the index of that tensor among the operation's outputs.


### Placing operations on different devices
[to do]

### Tensor-like objects
Many TensorFlow operations will accept a tensor-like object in place of a ```tf.Tensor```, and implicitly convert it to a ```tf.Tensor``` using the ```tf.convert_to_tensor``` method. Tensor-like objects include elements of the following types:
- tf.Tensor
- tf.Variable
- numpy.ndarray
- list
- Sclar python types: bool, float, int, str

You can register additional tensor-like types using **tf.register_tensor_conversion_function**.

## tf.Session
TensorFlow uses the **tf.Session** class to represent a connection between the client program---typically a Python program, although a similar interface is available in other languages---and the C++ runtime.

### Creating a ```tf.session```
```python
with tf.Session() as sess:
    # ...
```
Use context manager (with block) that can automatically close the session.  
**tf.Session.init** accepts three optional arguments:
- target. You may specify a ```grpc://``` URL to specify the address of a TensorFlow server
- graph. You can specify an explicit ```tf.Graph``` when you construct the session.
- config. You can specify a ```tf.ConfigProto``` that controls the behavior of the session.

### Using ```tf.Session.run``` to execute operations
The ```tf.Session.run``` method is the main mechanism for running a ```tf.Operation``` or evaluating a ```tf.Tensor```.
- ```tf.Session.run``` requires specifying a list of fetches, which determine the return values.
- ```tf.Session.run``` also optionally takes a dictionary of feeds, a mapping from ```tf.Tensor``` objects to values.
- ```tf.Session.run``` also accepts an optional options argument that enables you to specify options about the call, and an optional ```run_metadata``` argument that enables you to collect metadata about the execution. 

### Visualizing graph
The graph visualizer is a component of TensorBoard that renders the structure of your graph visually in a browser.

### Programming with multiple graphs
For many applications, a single graph is sufficient, TensorFlow also provides methods for manipulating the default graph. For example:
- A **tf.Graph** defines the namespace for **tf.Operation** objects: each operation in a single graph must have a unique name.
- The default graph stores information about every **tf.Operation** and **tf.Tensor** that was ever added to it.

You can install a different **tf.Graph** as the default graph, using the **tf.Graph.as_default** context manager:
```python
g_1 = tf.Graph()
with g_1.as_default():
  # Operations created in this scope will be added to `g_1`.
  c = tf.constant("Node in g_1")
  # Sessions created in this scope will run operations from `g_1`.
  sess_1 = tf.Session()

g_2 = tf.Graph()
with g_2.as_default():
  # Operations created in this scope will be added to `g_2`.
  d = tf.constant("Node in g_2")

# Alternatively, you can pass a graph when constructing a session:
# `sess_2` will run operations from `g_2`.
sess_2 = tf.Session(graph=g_2)
```

To inspect the current default graph, call **tf_get_default_graph**, which returns a **tf.Graph** object:
```python
# Print all of the operations in the default graph.
g = tf.get_default_graph()
print(g.get_operations())
```